<h1>Week 3 - Capstone</h1>

<h2>Import data into a dataframe.</h2>

In [ ]:
import pandas as pd

# Put csv of neighborhoods into "neighborhoods" variable
neighborhoods = pd.read_csv("toronto_neighborhoods.csv")

# Check that it worked
print(neighborhoods.head())

# Remove any with "not assigned"
neighborhoods = neighborhoods[neighborhoods.borough != 'Not assigned']

print(neighborhoods.shape)

<h2>Add coordinates.</h2>

In [ ]:
### I couldn't get Geocoder to work so I merged the provided csv with the neighborhood data.

coordinates = pd.read_csv('Geospatial_Coordinates.csv')

# Joins coordinates and neighborhoods dataframes
neighborhoods = coordinates.merge(neighborhoods, on='postal_code')

<h2>Install/import libraries that will be needed later.</h2>

In [ ]:
import numpy as np

# To work with JSON files
import json

!conda install -c conda-forge geopy --yes # only needed to run once
from geopy.geocoders import Nominatim

import requests # library to handle requests

# Import and set pandas display options, JSON support
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# matplotlib plotting
import matplotlib.cm as cm
import matplotlib.colors as colors

# import kmeans
from sklearn.cluster import KMeans

# install and import folium for mapping
!conda install -c conda-forge folium=0.5.0 --yes # only needed to run once
import folium

<h2>Set up map & show all neighborhoods.</h2>

In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map 
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['borough'], neighborhoods['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
# display map
map_toronto

<h2>Set up Foursquare credentials.</h2>

In [ ]:
CLIENT_ID = 'PZ1XLTP2AE3UZCNEFM5ZVGAEIITCYBIWITPAZNTUAWMIF43C' # your Foursquare ID
CLIENT_SECRET = 'LOOLAGGD4WSUF42S4OVATNV1DGI1FERGONOGIQYWCZNE0FLZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

<h2>Set up the URL for Foursquare.</h2>

In [ ]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)
url # display URL

<h2>Function to get nearby venues for each neighborhood.</h2>

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h2>Use the function to get the neighborhoods, latitude, and longitude from 'neighborhoods'.</h2>

In [ ]:
toronto_venues = getNearbyVenues(names = neighborhoods['neighborhood'],
                                   latitudes = neighborhoods['latitude'],
                                   longitudes = neighborhoods['longitude']
                                  )

<h2>Just checking things & summarizing data.</h2>

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()
toronto_venues.groupby('Neighborhood').count()
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

<h2>Onehot encoding.</h2>

In [ ]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()
toronto_onehot.shape
# group by neighborhood and average the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
toronto_grouped.shape

<h2>Print list of neighborhoods with top 5 common venues in each.</h2>

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----" + hood + "----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

<h2>Function to put everything into a pandas dataframe; in descending order.</h2>

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h2>Create new dataframe and display top 10 venues for each neighborhood.</h2>

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()




<h2>Start clustering the neighborhoods.</h2>

In [ ]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# I made all of my column names lowercase, so making "neighborhood" capitalized to be able to merge with neighborhoods_venues_sorted
toronto_merged.rename(columns = {'neighborhood':'Neighborhood'}, inplace=True)
# merge toronto_merged with neighborhoods_venues_sorted to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# For some reason, there are NaNs, so removing those rows.
toronto_merged = toronto_merged.dropna()

toronto_merged.head() # check data

<h2>Create a map with the clusters.</h2>

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1], # Had to cast cluster as integer
        fill=True,
        fill_color=rainbow[int(cluster)-1], # Had to cast cluster as integer
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters